# Tutorial: Numpy simultions of 2D gas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
class GasSimulation:

    def __init__(self, N=100, L=10.0, dt=0.01, radius=0.2, temperature=1.0, mass=1.0, steps=500):
        """Initialize simulation by specifiying: No of gas particles (N), Box dimensions (L), time-step (dt), radius, tempertauture, mass and simulation steps"""

        self.N = N
        self.L = L
        self.dt = dt
        self.radius = radius
        self.temperature = temperature
        self.mass = mass
        self.steps = steps  # Total steps for simulation
        
        # Initialize positions on the left side
        self.positions = np.column_stack((np.random.uniform(0, L/2, N), np.random.uniform(0, L, N)))
        
        # Initialize velocities using Maxwell-Boltzmann distribution
        self.velocities = np.random.normal(0, np.sqrt(self.temperature / self.mass), (N, 2))

        # Storage for post-processing
        self.positions_history  = []
        self.velocities_history = []

    def update_positions(self):
        """Updates positions and handles wall collisions"""

        self.positions += self.velocities * self.dt
        
        # Find particles that hit the left or right walls
        wall_collisions = (self.positions - self.radius < 0) | (self.positions + self.radius > self.L)
        
        # Reverse velocity for colliding particles
        self.velocities[wall_collisions] *= -1

        # Keep positions inside the valid range
        self.positions = np.clip(self.positions, self.radius, self.L - self.radius)

    def run_simulation(self):
        """Runs the simulation and stores data for post-analysis"""
        
        for _ in range(self.steps):
            
            self.update_positions()
            self.positions_history.append(self.positions.copy())
            self.velocities_history.append(self.velocities.copy())

In [ ]:
def animate_simulation(positions_history, L):

    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlim(0, L)
    ax.set_ylim(0, L)
    scatter = ax.scatter([], [], s=20, c='blue', alpha=0.6)

    def update(frame):
        scatter.set_offsets(positions_history[frame])
        return scatter,

    ani = animation.FuncAnimation(fig, update, frames=len(positions_history), interval=20, blit=True)
    
    return HTML(ani.to_jshtml())

In [ ]:
# Create and run simulation
sim = GasSimulation(N=100, steps=500)
sim.run_simulation()

# Animate the simulation
animate_simulation(sim.positions_history, sim.L)

## Project 1: Plot velocity and particle distributions

In [ ]:
#plot_speed_distribution(sim.velocities_history)
#plot_position_heatmap(sim.positions_history)

## Project 2: Implement elastic collisions between gas particles

### 1. Conservation Laws

When two particles collide elastically, both **momentum** and **kinetic energy** are conserved.

#### **Momentum Conservation**
$$
m_1 v_{1,\text{init}} + m_2 v_{2,\text{init}} = m_1 v_{1,\text{final}} + m_2 v_{2,\text{final}}
$$

#### **Kinetic Energy Conservation**
$$
\frac{1}{2} m_1 v_{1,\text{init}}^2 + \frac{1}{2} m_2 v_{2,\text{init}}^2 = \frac{1}{2} m_1 v_{1,\text{final}}^2 + \frac{1}{2} m_2 v_{2,\text{final}}^2
$$

where:
- $v_{1,\text{init}}$, $v_{2,\text{init}}$ are the initial speeds of particles 1 and 2.
- $v_{1,\text{final}}$, $v_{2,\text{final}}$ are the final speeds after the collision.



### 2. Collision Setup

For two **colliding particles**, we define:

- **Positions:** $ \mathbf{r}_1, \mathbf{r}_2 $
- **Velocities (before collision):** $ \mathbf{v}_{1,\text{init}}, \mathbf{v}_{2,\text{init}} $
- **Masses:** $ m_1, m_2 $

The **relative velocity** between the two particles before the collision is:

$$
\mathbf{v}_{\text{rel}} = \mathbf{v}_{1,\text{init}} - \mathbf{v}_{2,\text{init}}
$$

The **unit vector along the collision axis** (the direction along the line joining the particle centers) is:

$$
\hat{\mathbf{r}} = \frac{\mathbf{r}_1 - \mathbf{r}_2}{|\mathbf{r}_1 - \mathbf{r}_2|}
$$



### 3. Velocities After Collision

After the collision, the velocity component **along the collision axis** changes, while the **perpendicular components remain unchanged**.

Using **momentum** and **energy** conservation, the updated velocities are:

$$
\mathbf{v}_{1,\text{final}} = \mathbf{v}_{1,\text{init}} - \frac{2 m_2}{m_1 + m_2} (\mathbf{v}_{\text{rel}} \cdot \hat{\mathbf{r}}) \hat{\mathbf{r}}
$$

$$
\mathbf{v}_{2,\text{final}} = \mathbf{v}_{2,\text{init}} + \frac{2 m_1}{m_1 + m_2} (\mathbf{v}_{\text{rel}} \cdot \hat{\mathbf{r}}) \hat{\mathbf{r}}
$$

where:
- $ \mathbf{v}_{\text{rel}} \cdot \hat{\mathbf{r}} $ is the projection of **relative velocity** along the collision axis.
- Projection is **negative** when particles moving **towards each other** and positive if moving away from each other.
- The prefactors $ \frac{2 m_2}{m_1 + m_2} $ and $ \frac{2 m_1}{m_1 + m_2} $ ensure **momentum and energy conservation**.




### Need to update positions by checking for collisions

```python
        for i in range(self.N):
            for j in range(i + 1, self.N):

                r_rel = self.positions[i] - self.positions[j]    # Relative position
                v_rel = self.velocities[i] - self.velocities[j]  # Relative velocity

                dist = np.linalg.norm(r_rel)

                if dist < 2 * self.radius:  # Collision condition
                    # Fill out code
                    # Ensure that particles moving towards each other and not away from each other to avoid extra computation


```